In [ ]:
# =======================
# 1. Import Libraries
# =======================
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# =======================
# 2. Paths to Dataset
# =======================
# Use Kaggle dataset: "chest-xray-pneumonia"
# Folder structure should be:
# chest_xray/
#    train/
#    val/
#    test/

train_dir = "/content/chest_xray/train"
val_dir   = "/content/chest_xray/val"
test_dir  = "/content/chest_xray/test"

# =======================
# 3. Data Preprocessing
# =======================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary"
)

val_data = test_val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary"
)

test_data = test_val_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=False
)

# =======================
# 4. CNN Model
# =======================
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# =======================
# 5. Callbacks
# =======================
checkpoint = ModelCheckpoint("best_pneumonia_model.h5", save_best_only=True)
earlystop = EarlyStopping(patience=5, restore_best_weights=True)

# =======================
# 6. Train Model
# =======================
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[checkpoint, earlystop]
)

# =======================
# 7. Evaluate Model
# =======================
loss, acc = model.evaluate(test_data)
print("Test Accuracy:", acc)

# =======================
# 8. Save Model
# =======================
model.save("pneumonia_xray_model.h5")